## Base Cloud ServiceHandler

In [14]:
#| default_exp services.cloud.base

In [15]:
#| exporti

from dc_logger.client.base import LogEntry
from dc_logger.client.exceptions import LogWriteError

In [16]:
#| exporti
from dc_logger.client.base import ServiceHandler, ServiceConfig

import socket
from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import List, Dict, Any

In [17]:
#| export

@dataclass
class CloudServiceConfig(ServiceConfig):
    """stores auth and connection information to a service provider"""
    cloud_provider : str

    @abstractmethod
    def to_platform_config(self) -> Dict[str, Any]:
        """Get cloud provider specific configuration"""
        raise NotImplementedError()
    
@dataclass
class CloudHandler(ServiceHandler):
    """base class for communicating with service provider (route functions)"""

    # def __init__(self, config):
    #     super().__init__(config)
    #     self.cloud_config = config.to_platform_config()
    #     config.validate_config()

    """should i pool logs"""

    async def _send_to_cloud(self, entries: List[LogEntry]) -> bool:
        """Send log entries to Datadog using direct HTTP API"""

        def submit_logs():
            return self._send_logs_simple_api(entries)

        with concurrent.futures.ThreadPoolExecutor() as executor:
            future = executor.submit(submit_logs)
            result = await asyncio.wrap_future(future)
            return result

    async def _write_pooling(self, entry):
        # if pool:
        #     await self._send_to_cloud(self.buffer)
        pass

    async def write(self, entry) -> bool:
        """Write entries to cloud provider"""
        try:
            return await self._write_pooling(entry)
        except Exception as e:
            raise LogWriteError(f"Error sending logs to cloud provider: {e}")

    def _get_hostname(self) -> str:
        """Get the actual hostname/IP address of the machine"""
        try:
            # Try to get the hostname, fallback to IP if needed
            hostname = socket.gethostname()
            # Get the IP address for more specific identification
            ip_address = socket.gethostbyname(hostname)
            return ip_address
        except:
            # Fallback to localhost if hostname resolution fails
            return "127.0.0.1"


In [18]:
#| hide
import nbdev; nbdev.nbdev_export('./base.ipynb')